
# Qwen2.5‑VL Forward Pass Demo (Cosmic‑AI Style, **No `pipeline`**)

This notebook shows end‑to‑end usage of **Qwen/Qwen2.5‑VL‑7B‑Instruct** with **direct model calls** (no `transformers.pipeline`).  
You'll learn how to:
- Load `AutoProcessor` and `Qwen2_5_VLForConditionalGeneration`,
- Build a multimodal **conversation** (images + text) by hand,
- Run a **pure forward pass** to inspect logits,
- Run **generation** without `pipeline`,
- Try astronomy‑style tasks (nebulae, galaxy morphology, HR diagram, light curves),
- Do a tiny rough lexical comparison to a reference caption.

> **Note:** You need local images (see the `astro/` folder hints below).


In [1]:

# 0) Install dependencies (uncomment and run once)
!pip install -U transformers accelerate torch torchvision pillow
# Optional: flash-attn for supported GPUs
# !pip install flash-attn --no-build-isolation


  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 1.1 MB/s  0:00:10 eta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 13.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 23.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 90.7 kB/s  0:00:50 eta 0:00:14
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 20.0 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 MB 2.2 MB/s  0:00:37 eta 0:00:020m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 26.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 26.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━

In [2]:

# 1) Imports, device, and model choice
import torch
from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration

MODEL_ID = "Qwen/Qwen2.5-VL-7B-Instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = (
    torch.bfloat16 if (device == "cuda" and torch.cuda.is_bf16_supported())
    else torch.float16 if device == "cuda"
    else torch.float32
)

processor = AutoProcessor.from_pretrained(MODEL_ID, trust_remote_code=True)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    MODEL_ID,
    torch_dtype=dtype,
    device_map="auto" if device == "cuda" else None,
    attn_implementation="flash_attention_2" if device == "cuda" else None
)
model.eval()

print(f"Device: {device}  |  DType: {dtype}")


/Users/wilku/Desktop/UT_Workspace/Research/CosmicAI/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/wilku/Desktop/UT_Workspace/Research/CosmicAI/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
`torch_dtype` is d

KeyboardInterrupt: 


## 2) Astronomy images to try
Place one or more images in a local folder, e.g. `./astro/`:
- `sdss_spiral.jpg` (spiral galaxy)
- `orion_nebula.jpg` (nebula / star‑forming region)
- `jwst_mirror.jpg` (instrument photo)
- `hr_diagram.png` (scientific chart)
- `sn_lightcurve.png` (supernova light curve)

Then adjust the file paths below.


In [ ]:

# 3) Build a chat turn (image + text) manually
conversation = [{
    "role": "user",
    "content": [
        {"type": "image", "path": "./astro/orion_nebula.jpg"},
        {
            "type": "text",
            "text": "Identify the main regions (e.g., ionization fronts, dark dust lanes). "
                    "What physical processes are likely happening here?"
        }
    ]
}]
print(conversation)


In [ ]:

# 4) Convert raw inputs -> tensors via the processor (no pipeline)
inputs = processor.apply_chat_template(
    conversation,
    tokenize=True,
    return_dict=True,
    add_generation_prompt=True,   # adds assistant role for decoding to continue
    return_tensors="pt",
)

# Move tensors to the model device (avoid CPU/GPU mismatch)
inputs = {k: (v.to(model.device) if torch.is_tensor(v) else v) for k, v in inputs.items()}

# Peek at the tensor shapes/keys
shape_summary = {k: (tuple(v.shape) if torch.is_tensor(v) else str(type(v))) for k, v in inputs.items()}
shape_summary


In [ ]:

# 5) Pure forward pass (logits only) — no decoding here
with torch.inference_mode():
    out = model(**inputs)     # calls forward(...)
    logits = out.logits       # [batch, seq_len, vocab_size]

print("Logits shape:", tuple(logits.shape))

# Inspect the next-token distribution for the last position
last_step = logits[0, -1]
probs = last_step.softmax(dim=-1)
topk = torch.topk(probs, k=10)
{
    "top10_token_ids": topk.indices.tolist(),
    "top10_probs": [float(x) for x in topk.values]
}


In [ ]:

# 6) Generation (still no pipeline): greedy decoding for determinism
with torch.inference_mode():
    gen_ids = model.generate(
        **inputs,
        max_new_tokens=192,
        do_sample=False,       # greedy
        temperature=1.0,
    )

response = processor.batch_decode(gen_ids, skip_special_tokens=True)[0]
print(response)


In [ ]:

# 7) Tiny rough lexical comparison to a reference caption (toy metric)
reference = (
    "This is the Orion Nebula (M42), a star-forming region with prominent ionization fronts, "
    "bright emission from hydrogen, and dark dust lanes. Young stellar objects and proplyds are common."
)

import re
def rough_compare(pred: str, ref: str):
    tok = lambda s: set(re.findall(r"[a-zA-Z]+", s.lower()))
    a, b = tok(pred), tok(ref)
    precision = len(a & b) / (len(a) + 1e-9)
    recall    = len(a & b) / (len(b) + 1e-9)
    f1 = 2*precision*recall/(precision+recall+1e-9)
    return {"precision": float(precision), "recall": float(recall), "f1": float(f1)}

rough_compare(response, reference)


In [ ]:

# 8) Visual Question Answering (VQA) example — galaxy morphology
conversation = [{
    "role": "user",
    "content": [
        {"type": "image", "path": "./astro/sdss_spiral.jpg"},
        {"type": "text",  "text": "Is this galaxy spiral, barred spiral, elliptical, or irregular? Justify your answer."}
    ]
}]
inputs = processor.apply_chat_template(conversation, tokenize=True, return_dict=True, add_generation_prompt=True, return_tensors="pt")
inputs = {k: (v.to(model.device) if torch.is_tensor(v) else v) for k, v in inputs.items()}
with torch.inference_mode():
    answer_ids = model.generate(**inputs, max_new_tokens=128, do_sample=False)
processor.batch_decode(answer_ids, skip_special_tokens=True)[0]


In [ ]:

# 9) Scientific chart reasoning — HR diagram or light curve
conversation = [{
    "role": "user",
    "content": [
        {"type": "image", "path": "./astro/hr_diagram.png"},
        {"type": "text",  "text": "Explain what this diagram represents and how color relates to temperature and luminosity."}
    ]
}]
inputs = processor.apply_chat_template(conversation, tokenize=True, return_dict=True, add_generation_prompt=True, return_tensors="pt")
inputs = {k: (v.to(model.device) if torch.is_tensor(v) else v) for k, v in inputs.items()}
with torch.inference_mode():
    chart_ids = model.generate(**inputs, max_new_tokens=160, do_sample=False)
processor.batch_decode(chart_ids, skip_special_tokens=True)[0]
